In [1]:
pip install bertopic

Note: you may need to restart the kernel to use updated packages.


This is just installing the relevant python libraries 

In [2]:
from bertopic import BERTopic
import pandas as pd
import datetime 

### reading the metakaggle dataset 

In [3]:
kernels = pd.read_csv("../input/meta-kaggle/Kernels.csv")

This table has got CurrentUrlSlug which is basically a proxy for the notebook title hence we'll use this for Topic Modelling

In [4]:
kernels.head()

,Id,AuthorUserId,CurrentKernelVersionId,ForkParentKernelVersionId,ForumTopicId,FirstKernelVersionId,CreationDate,EvaluationDate,MadePublicDate,IsProjectLanguageTemplate,CurrentUrlSlug,Medal,MedalAwardDate,TotalViews,TotalComments,TotalVotes
0,1,2505,205.0,NaN,NaN,1.0,03/25/2015 18:25:32,03/23/2018,03/25/2015,False,hello,NaN,NaN,263,0,0
1,2,3716,1748.0,NaN,26670.0,2.0,03/25/2015 18:31:07,04/16/2015,03/25/2015,False,rf-proximity,3.0,07/15/2016,8778,1,13
2,4,3716,41.0,NaN,NaN,9.0,03/25/2015 21:57:36,03/23/2018,03/25/2015,False,r-version,NaN,NaN,127,0,0
3,5,28963,19.0,NaN,NaN,13.0,03/25/2015 22:01:04,03/23/2018,03/25/2015,False,test1,NaN,NaN,122,0,0
4,6,3716,21.0,NaN,NaN,15.0,03/25/2015 22:19:00,03/23/2018,03/25/2015,False,are-icons-missing,NaN,NaN,139,0,0


This filters our dataframe for the first half of 2022

In [5]:
kernels_2022 = kernels[(pd.to_datetime(kernels['CreationDate'])>=pd.Timestamp(2022,1,1)) & (pd.to_datetime(kernels['CreationDate'])<=pd.Timestamp(2022,6,30))]

In [6]:
kernels_2022.shape

(121136, 16)

This takes out the notebooks that have been forked, notebooks with 0 views too are taken out 

In [7]:
kernel_2022_nofork_gt_0views = kernels_2022[~kernels_2022.ForkParentKernelVersionId.notnull() & kernels_2022.TotalViews > 0]

In [8]:
kaggle_notebook_titles_2022 = list(kernel_2022_nofork_gt_0views.CurrentUrlSlug) #using the relevant column 

In [9]:
topic_model = BERTopic(verbose=True, embedding_model="paraphrase-MiniLM-L12-v2", min_topic_size=50,  calculate_probabilities=True)

In [10]:
topics, probs = topic_model.fit_transform(kaggle_notebook_titles_2022)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.70k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/631 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

2022-06-19 17:14:14,301 - BERTopic - Transformed documents to Embeddings


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2022-06-19 17:15:11,257 - BERTopic - Reduced dimensionality


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

2022-06-19 17:16:18,443 - BERTopic - Clustered reduced embeddings


In [11]:
freq = topic_model.get_topic_info()

In [12]:
freq.head(6) #get the 5 most common topics

,Topic,Count,Name
0,-1,18978,-1_movie_720p_hindi_best
1,0,3826,0_game_download_sex_pc
2,1,2393,1_pdf_book_ebook_urdu
3,2,2224,2_de_solucionario_la_descargar
4,3,1825,3_crack_cracked_pro_full
5,4,1725,4_notebook_notebook2_mbp_laptops


In [13]:
topic_model.get_topic(0)

[('game', 0.01619226197719404),
 ('download', 0.014829659616033641),
 ('sex', 0.014163600976191987),
 ('pc', 0.013362302249101612),
 ('the', 0.012238726624459363),
 ('version', 0.011817203943649691),
 ('of', 0.01097697500082065),
 ('full', 0.010846212872529765),
 ('free', 0.010609108701790597),
 ('mod', 0.008769764866878324)]

In [14]:
topic_model.visualize_topics()

In [15]:
topic_model.visualize_barchart()